In [1]:
%matplotlib inline
import pandas as pd
from skbio import DistanceMatrix
from os.path import join
from skbio.stats.ordination import pcoa
from q2d2 import rarify

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
from IPython.parallel import Client
clients = Client(profile='data-analysis-conda')
dview = clients.direct_view()

In [3]:
home = '/home/office-microbe-files'
notebooks = '/home/johnchase/office-project/office-microbes/notebooks'

Load mapping file
----------
Load the mapping file and filter it to contain only 16S

In [4]:
map_fp = join(home, 'master_map_150908.txt')
sample_md = pd.read_csv(map_fp, sep='\t', index_col=0, dtype=str)
sample_md = sample_md[sample_md['16SITS'] == '16S']

Load the full unrarefied OTU table
--------------------------

In [5]:
table_fp = join(home, 'pick_otus_out_97/otu_table_mc2_w_tax_no_pynast_failures.txt')
table = pd.read_csv(table_fp, sep='\t', skiprows=1, index_col=0, dtype="object")
table.index = table.index.astype(str)
table = table.astype('float')

Filter OTU table to be 1-3 and 2-3 samples only
-------------------
The first thing to do is to filter the OTU tables down into two different OTU tables, one that contains just run 1 and 2 samples, and another that only contains run 1 and 3. This is a bit weird, but these are the only combinations that have techincal replicates, and they don't overlap...

In [6]:
run_1_3_ids = sample_md[(sample_md["Run"] == "1") | (sample_md["Run"] == "3")].index
run_2_3_ids = sample_md[(sample_md["Run"] == "2") | (sample_md["Run"] == "3")].index

There are many samples that are missing from the OTU table and I really don't understand why. There are other samples that are not missing with 0 sequences, so that shouldn't be the problem.  
This is why there is the extra step if checking that the IDs are in the table before filtering

In [7]:
table_13 = table[run_1_3_ids[run_1_3_ids.isin(table.columns)]]
table_23 = table[run_2_3_ids[run_2_3_ids.isin(table.columns)]]

These don't take too long to run, but others later will take quite a while so I'm defining a function here to run this in parallel

In [8]:
def df_to_file(df, filepath):
    df.to_csv(filepath, sep='\t')

In [9]:
sample_fps = [join(notebooks, i) for i in ['table_13.txt', 'table_23.txt']]
dview.map(df_to_file, [table_13, table_23], sample_fps)

<AsyncMapResult: df_to_file>

Find most abundant OTUs and rarefy the tables
------------------
While some of the OTUs may be overlappping in blanks the top ten of each will be filtered out, even if this means that only 17 of of 20 are unique for instance

In [10]:
def get_ids(sample_metadata, otu_table, run):
    run_ids = sample_metadata[(sample_metadata['Run'] == run) & (sample_metadata['OfficeSample'] == 'no')].index
    run_otus = otu_table[list(set(run_ids) & set(otu_table.columns))].sum(axis=1)
    run_otus = run_otus.sort(ascending=False, inplace=False).index.tolist()
    return run_otus

In [11]:
run_1_otus = get_ids(sample_md, table, "1")
run_2_otus = get_ids(sample_md, table, "2")
run_3_otus = get_ids(sample_md, table, "3")

/home/johnchase/.conda/envs/data-analysis/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


In [12]:
table_13_otus_10 = rarify(table_13.drop(run_1_otus[:10] + run_3_otus[:10], inplace=False), 1000)
table_13_otus_100 = rarify(table_13.drop(run_1_otus[:100] + run_3_otus[:100], inplace=False), 1000)
table_13_otus_1000 = rarify(table_13.drop(run_1_otus[:1000] + run_3_otus[:1000], inplace=False), 1000)

In [13]:
table_23_otus_10 = rarify(table_23.drop(run_2_otus[:10] + run_3_otus[:10], inplace=False), 1000)
table_23_otus_100 = rarify(table_23.drop(run_2_otus[:100] + run_3_otus[:100], inplace=False), 1000)
table_23_otus_1000 = rarify(table_23.drop(run_2_otus[:1000] + run_3_otus[:1000], inplace=False), 1000)

In [14]:
!mkdir blank_filtered_tables

In [16]:
otus = [10, 100, 1000, 10, 100, 1000]
runs = ['13']*3 + ['23']*3
paths = [join(notebooks, 'blank_filtered_tables/table_{0}_otus_{1}.txt'.format(run, otu)) for run, otu in zip(runs, otus)]
dfs = [table_13_otus_10, table_13_otus_100, table_13_otus_1000, 
       table_23_otus_10, table_23_otus_100, table_23_otus_1000]

h = dview.map(df_to_file, dfs, paths)

Convert all of the tables to biom to make them easier to work with
----------------------

```bash
biom convert -i table_13.txt -o table_13.biom --to-hdf5 --table-type "OTU table"&  
biom convert -i table_23.txt -o table_23.biom --to-hdf5 --table-type "OTU table"& 
biom convert -i blank_filtered_tables/table_13_otus_10.txt -o blank_filtered_tables/table_13_otus_10.biom --to-hdf5 --table-type "OTU table"&  
biom convert -i blank_filtered_tables/table_13_otus_100.txt -o blank_filtered_tables/table_13_otus_100.biom --to-hdf5 --table-type "OTU table"&
biom convert -i blank_filtered_tables/table_13_otus_1000.txt -o blank_filtered_tables/table_13_otus_1000.biom --to-hdf5 --table-type "OTU table"&  
biom convert -i blank_filtered_tables/table_23_otus_10.txt -o blank_filtered_tables/table_23_otus_10.biom --to-hdf5 --table-type "OTU table" & 
biom convert -i blank_filtered_tables/table_23_otus_100.txt -o blank_filtered_tables/table_23_otus_100.biom --to-hdf5 --table-type "OTU table"&
biom convert -i blank_filtered_tables/table_23_otus_1000.txt -o blank_filtered_tables/table_23_otus_1000.biom --to-hdf5 --table-type "OTU table"

rm blank_filtered_tables/table_*_otus_*.txt
```


```bash
#SBATCH --job-name=bd_all
#SBATCH --output=/scratch/jc33/bdiv/std_out.txt
#SBATCH --error=/scratch/jc33/bdiv/std_err.txt
#SBATCH --workdir=/scratch/jc33/beta_div
#SBATCH --mail-type=ALL
#SBATCH --time=2-00:00:00
#SBATCH --mem-per-cpu=32000

module load qiime

srun parallel_beta_diversity.py -i /scratch/jc33/bdiv/table_13.biom -o /scratch/jc33/bdiv/bdiv_13 -t /scratch/jc33/bdiv/rep_set.tre &
parallel_beta_diversity.py -i /scratch/jc33/bdiv/table_12.biom -o /scratch/jc33/bdiv/bdiv_12 -t /scratch/jc33/bdiv/rep_set.tre &

parallel_beta_diversity.py -i /scratch/jc33/bdiv/blank_filtered_tables/table_13_otus_10.biom -o /scratch/jc33/bdiv/blank_filtered_tables/bdiv_13_10_out -t /scratch/jc33/beta_div/rep_set.tre &
parallel_beta_diversity.py -i /scratch/jc33/bdiv/blank_filtered_tables/table_13_otus_100.biom -o /scratch/jc33/bdiv/blank_filtered_tables/bdiv_13_100_out -t /scratch/jc33/beta_div/rep_set.tre &
parallel_beta_diversity.py -i /scratch/jc33/bdiv/blank_filtered_tables/table_13_otus_1000.biom -o /scratch/jc33/bdiv/blank_filtered_tables/bdiv_13_1000_out -t /scratch/jc33/beta_div/rep_set.tre &
parallel_beta_diversity.py -i /scratch/jc33/bdiv/blank_filtered_tables/table_23_otus_10.biom -o /scratch/jc33/bdiv/blank_filtered_tables/bdiv_23_10_out -t /scratch/jc33/beta_div/rep_set.tre &
parallel_beta_diversity.py -i /scratch/jc33/bdiv/blank_filtered_tables/table_23_otus_100.biom -o /scratch/jc33/bdiv/blank_filtered_tables/bdiv_23_100_out -t /scratch/jc33/beta_div/rep_set.tre &
parallel_beta_diversity.py -i /scratch/jc33/bdiv/blank_filtered_tables/table_23_otus_1000.biom -o /scratch/jc33/bdiv/blank_filtered_tables/bdiv_23_1000_out -t /scratch/jc33/beta_div/rep_set.tre 
```